In [1]:
# Parameters
artificial_humans = "../../data/training/ah_10/data/model.pt"
artificial_humans_model = "graph"
output_path = "../../data/manager/v2/dev/"
model_args = {
    "hidden_size": 5,
    "add_rnn": True,
    "add_edge_model": True,
    "add_global_model": False,
    "x_encoding": [
        {"name": "prev_contributions", "n_levels": 21, "encoding": "numeric"},
        {"name": "prev_punishments", "n_levels": 31, "encoding": "numeric"},
    ],
    "u_encoding": [
        {"name": "round_number", "n_levels": 16, "encoding": "numeric"},
        {"name": "prev_common_good", "norm": 128, "etype": "float"},
    ],
}
opt_args = {"lr": 0.003}
gamma = 1.0
eps = 0.2
target_update_freq = 20
n_episode_steps = 16
n_episodes = 100
memory_size = 50
sample_args = {"batch_size": 10}
n_test_episodes = 10
eval_freq = 10
device = "cpu"


In [2]:
%load_ext autoreload
%autoreload 2

import torch as th
import os
from itertools import count

from aimanager.manager.memory import Memory
from aimanager.manager.environment import ArtificialHumanEnv
from aimanager.artificial_humans import AH_MODELS
from aimanager.manager.manager import ArtificalManager
from aimanager.manager.graph_memory import GraphMemory
from aimanager.manager.evaluation import ManagerEvaluator

from torch_geometric.data import Batch


output_file =  os.path.join(output_path, 'data', "eval")


/home/mpib/brinkmann/repros/algorithmic-institutions/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mpib/brinkmann/repros/algorithmic-institutions/.venv/lib/python3.9/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [17]:
device = th.device(device)
artifical_humans = AH_MODELS[artificial_humans_model].load(artificial_humans).to(device)

env = ArtificialHumanEnv(
    artifical_humans=artifical_humans, n_agents=4, n_contributions=21, n_punishments=31, episode_steps=n_episode_steps, device=device)

manager = ArtificalManager(
    n_contributions=21, n_punishments=31, model_args=model_args, opt_args=opt_args, gamma=gamma, 
    target_update_freq=target_update_freq, default_values=artifical_humans.default_values, device=device)

replay_mem = GraphMemory(n_episodes=memory_size, n_episode_steps=n_episode_steps, n_nodes=4, device=device)
recorder = Memory(n_episodes=n_episodes, n_episode_steps=n_episode_steps, device=device)
evaluator = ManagerEvaluator(n_episodes, n_test_episodes, eval_freq, n_episode_steps, output_file)

display_freq = n_episodes // 20

for episode in range(n_episodes):
    if (episode > 1) and (episode % display_freq == 0):
        avg_common_good = recorder.last(display_freq)['common_good'].mean()
        avg_reward = replay_mem.last(display_freq)['reward'].mean()
        print(f'Start episode {episode}. Avg common good {avg_common_good}. Avg reward {avg_reward}')

    state = env.init_episode()

    manager.init_episode(episode)

    for step in count():
        state_ = {k: v.unsqueeze(0).unsqueeze(-1) for k, v in state.items()}
        obs = Batch.from_data_list(manager.encode(state_, edge_index=env.edge_index))

        # Get q values from controller
        q_values = manager.get_q(manager_observations=obs, first=step == 0).squeeze(1)

        # Sample a action
        selected_action = manager.eps_greedy(q_values=q_values, eps=eps)

        state = env.punish(selected_action)
        recorder.add(**state, episode_step=step)

        # pass actions to environment and advance by one step
        state, reward, done = env.step()
        replay_mem.add(
            action=selected_action, reward=reward, 
            obs=obs)
 
        if done:
            replay_mem.next_episode(episode)
            
            # allow manager to update itself
            sample = replay_mem.sample(**sample_args)
            

            if sample is not None:
                manager.update(**sample)
            break
    recorder.add(episode_step=step, **state)
    recorder.next_episode(episode)
    evaluator.eval_episode(manager, env, episode)


Start episode 5. Avg common good 24.84250259399414. Avg reward -0.877273678779602
Start episode 10. Avg common good 23.104999542236328. Avg reward -0.9243236780166626
Start episode 15. Avg common good 49.01250076293945. Avg reward -0.2702736258506775
Start episode 20. Avg common good 46.790000915527344. Avg reward -0.39632365107536316
Start episode 25. Avg common good 37.215003967285156. Avg reward -0.4773736894130707


KeyboardInterrupt: 